### Init setup

In [1]:
## For google colab only
from google.colab import drive
drive.mount('/content/drive')
import sys, os
## on Phuc's drive
%cd /content/drive/MyDrive/Uni/Thesis/Repo 
os.chdir("/content/drive/MyDrive/Uni/Thesis/Repo")
!pip install ipython-autotime
%load_ext autotime

Mounted at /content/drive
/content/drive/MyDrive/Uni/Thesis/Repo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 47.8 MB/s 
time: 445 µs (started: 2022-11-21 01:39:37 +00:00)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch, time, shutil, cv2, sys, os, json
import torch.nn as nn
from skimage import io
import torchvision
import pandas as pd
from pandas.io.json import json_normalize
from tqdm.notebook import tqdm
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from albumentations.pytorch.transforms import ToTensorV2
from torchvision import utils
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize,Flip,
                            Compose, GaussNoise)

time: 1.88 s (started: 2022-11-18 07:40:57 +00:00)


### Verify GPU and driver

In [ ]:
!nvidia-smi

Fri Nov 18 07:40:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Caffe setup

### Get Caffe

In [ ]:
!sudo apt install caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
caffe-cuda is already the newest version (1.0.0-6build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
time: 2.06 s (started: 2022-11-18 07:40:59 +00:00)


In [ ]:
!git clone https://github.com/BVLC/caffe.git
!git reset --hard 9b891540183ddc834a02b2bd81b31afae71b2153 #reset to the newest revision that worked OK on 27.03.2021

fatal: destination path 'caffe' already exists and is not an empty directory.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
time: 264 ms (started: 2022-11-18 07:41:01 +00:00)


### Install prepare dependencies

In [ ]:
# !apt install -y caffe-cuda

time: 508 µs (started: 2022-11-18 07:41:01 +00:00)


In [ ]:
# !sudo apt-cache search libhdf5-
# !sudo apt-cache search gflags
# !sudo apt --fix-broken install
!sudo apt-get install libgflags2.2 
!sudo apt-get install libgflags-dev
!sudo apt-get install libgoogle-glog-dev
# !sudo apt-get install libhdf5-10 - replaced with 100
!sudo apt-get install libhdf5-100
!sudo apt-get install libhdf5-serial-dev
!sudo apt-get install libhdf5-dev
# !sudo apt-get install libhdf5-cpp-11 - replaced with 100
!sudo apt-get install libhdf5-cpp-100
!sudo apt-get install libprotobuf-dev protobuf-compiler

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgflags2.2 is already the newest version (2.2.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgflags-dev is already the newest version (2.2.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgoogle-glog-dev is already the newest version (0.3.5-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove i

In [ ]:
!find /usr -iname "*hdf5.so"
# got: /usr/lib/x86_64-linux-gnu/hdf5/serial
!find /usr -iname "*hdf5_hl.so"

/usr/lib/x86_64-linux-gnu/libhdf5.so
/usr/lib/x86_64-linux-gnu/hdf5/serial/libhdf5.so
/usr/lib/x86_64-linux-gnu/libhdf5_hl.so
/usr/lib/x86_64-linux-gnu/hdf5/serial/libhdf5_hl.so
time: 1.56 s (started: 2022-11-18 07:41:24 +00:00)


In [ ]:
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial.so /usr/lib/x86_64-linux-gnu/libhdf5.so
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial_hl.so /usr/lib/x86_64-linux-gnu/libhdf5_hl.so

ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libhdf5.so': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libhdf5_hl.so': File exists
time: 245 ms (started: 2022-11-18 07:41:26 +00:00)


In [ ]:
#!find /usr -iname "*hdf5.h*" # got:
# /usr/include/hdf5/serial/hdf5.h 
# /usr/include/opencv2/flann/hdf5.h
# Let's try the first one.
%env CPATH="/usr/include/hdf5/serial/"
#fatal error: hdf5.h: No such file or directory

env: CPATH="/usr/include/hdf5/serial/"
time: 1.62 ms (started: 2022-11-18 07:41:26 +00:00)


In [ ]:
!sudo apt-get install libleveldb-dev
!sudo apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev
!sudo apt-get install libsnappy-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libleveldb-dev is already the newest version (1.20-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgflags-dev is already the newest version (2.2.1-1).
libgoogle-glog-dev is already the newest version (0.3.5-1).
liblmdb-dev is already the newest version (0.9.21-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsnappy-dev is already the newest version (1.1.7-1).
The following pack

### Build caffe from source files

In [ ]:
!echo $CPATH
%cd caffe
!ls
!make clean
!cp Makefile.config.example Makefile.config

"/usr/include/hdf5/serial/"
/content/drive/MyDrive/Uni/Thesis/Repo/caffe
build		CONTRIBUTING.md  docs	     Makefile		      python
_caffe		CONTRIBUTORS.md  examples    Makefile.config	      README.md
caffe.cloc	data		 include     Makefile.config.example  scripts
cmake		distribute	 INSTALL.md  matlab		      src
CMakeLists.txt	docker		 LICENSE     models		      tools
time: 4 s (started: 2022-11-18 07:41:32 +00:00)


### Use sed for editing Makefile.config

In [ ]:
!sed -i 's/-gencode arch=compute_20/#-gencode arch=compute_20/' Makefile.config #old cuda versions won't compile 
!sed -i 's/\/usr\/local\/include/\/usr\/local\/include \/usr\/include\/hdf5\/serial\//'  Makefile.config #one of the 4 things needed to fix hdf5 issues
!sed -i 's/# OPENCV_VERSION := 3/OPENCV_VERSION := 3/' Makefile.config #We actually use opencv 4.1.2, but it's similar enough to opencv 3.
!sed -i 's/code=compute_61/code=compute_61 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_75,code=sm_75 -gencode=arch=compute_75,code=compute_75/' Makefile.config #support for new GPUs

time: 491 ms (started: 2022-11-18 07:41:36 +00:00)


In [ ]:
!make all -j 4 # -j would use all availiable cores, but RAM related errors occur

PROTOC src/caffe/proto/caffe.proto
NVCC src/caffe/layers/absval_layer.cu
NVCC src/caffe/layers/base_data_layer.cu
NVCC src/caffe/layers/accuracy_layer.cu
NVCC src/caffe/layers/batch_norm_layer.cu
NVCC src/caffe/layers/batch_reindex_layer.cu
NVCC src/caffe/layers/bias_layer.cu
NVCC src/caffe/layers/bnll_layer.cu
NVCC src/caffe/layers/clip_layer.cu
NVCC src/caffe/layers/concat_layer.cu
NVCC src/caffe/layers/contrastive_loss_layer.cu
NVCC src/caffe/layers/conv_layer.cu
NVCC src/caffe/layers/crop_layer.cu
NVCC src/caffe/layers/cudnn_conv_layer.cu
NVCC src/caffe/layers/cudnn_deconv_layer.cu
NVCC src/caffe/layers/cudnn_lcn_layer.cu
NVCC src/caffe/layers/cudnn_lrn_layer.cu
NVCC src/caffe/layers/cudnn_pooling_layer.cu
NVCC src/caffe/layers/cudnn_relu_layer.cu
NVCC src/caffe/layers/cudnn_sigmoid_layer.cu
NVCC src/caffe/layers/cudnn_softmax_layer.cu
NVCC src/caffe/layers/cudnn_tanh_layer.cu
NVCC src/caffe/layers/deconv_layer.cu
NVCC src/caffe/layers/dropout_layer.cu
NVCC src/caffe/layers/eltwise

### Verify

In [ ]:
cd ..

/content/drive/MyDrive/Uni/Thesis/Repo
time: 3.05 ms (started: 2022-11-18 07:49:26 +00:00)


In [ ]:
import cv2
print(cv2.__version__)
import caffe

4.6.0


ModuleNotFoundError: ignored

time: 133 ms (started: 2022-11-18 07:49:26 +00:00)


## Setup for MMSeg

In [ ]:
# To avoid non-essential warnings 
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

time: 66.6 ms (started: 2022-11-18 02:58:03 +00:00)


In [ ]:
""" PY FILES """
py_file_location = "/content/drive/MyDrive/Uni/Thesis/Repo/utils"
sys.path.append(os.path.abspath(py_file_location))

time: 626 µs (started: 2022-11-18 02:58:04 +00:00)


In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version
# Install PyTorch
!pip install torch==1.12.0 torchvision --extra-index-url https://download.pytorch.org/whl/cu113
# Install MMCV
!pip install openmim
!mim install mmcv-full==1.6.0
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |██████████████▌                 | 834.1 MB 1.2 MB/s eta 0:14:06tcmalloc: large alloc 1147494400 bytes == 0x65d24000 @  0x7f0d8d54f615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 237 kB 59.6 MB/s 
     |████████████████████████████████| 51 kB 7.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.1 MB 1.4 MB/s 
     |████████████████████████████████| 190 kB 20.4 MB/s 
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 12375, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 12375 (delta 3), reused 13 (delta 1), pack-reused 12351
Receiving objects: 100% (12375/12375), 14.96 MiB | 7.59 MiB/s, done.
Resolving deltas: 100% (8986/8986), done.
Checking out files: 100% (1238/1238), done.
/content/drive/MyDrive/Uni/Thesis

In [ ]:
from parse_dataset import *
from eval import *
from image_extract import *

load checkpoint from local path: /content/drive/MyDrive/Uni/Thesis/Repo/checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
time: 18.1 s (started: 2022-11-18 03:02:44 +00:00)


In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
# config_file = 'configs/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes.py'
# checkpoint_file = '../checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

load checkpoint from local path: /content/drive/MyDrive/Uni/Thesis/Repo/checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
time: 1.04 s (started: 2022-11-18 03:03:02 +00:00)


In [ ]:
detectDS_link = '/content/drive/MyDrive/Uni/Thesis/dataset/detection'
# classifyDS_link = '/content/drive/MyDrive/Uni/Thesis/dataset/classification'
# In detection folder
dtecDS_info = "/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees/detection_datasets.json"
#clsDS_info = "/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees/classification_datasets.json"
rois_info =  "/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees/rois.json"
pano_info = "/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees/panos.json"

time: 837 µs (started: 2022-11-18 03:03:03 +00:00)


In [ ]:
cd ..

/content/drive/MyDrive/Uni/Thesis/Repo
time: 2.88 ms (started: 2022-11-18 03:03:03 +00:00)


## Main part of code

In [ ]:
dsDir = '/content/drive/MyDrive/Uni/Thesis/dataset/detection/PasadenaUrbanTrees'
pDs = ParseDataset(dsDir)
with open(dtecDS_info) as json_data:
  data = json.load(json_data)

time: 6.03 s (started: 2022-11-18 03:04:47 +00:00)


In [ ]:
def getDtecDFInfo(ds):
    dsX = pd.DataFrame(list(ds['extended']))
    df = pd.concat([ds,dsX], axis=1)
    # df = df.drop(['Longitude','Latitude','extended'], axis=1)

    return df
dtecDF_Itrain, dtecDF_Itest, dtecDF_Ival = [getDtecDFInfo(pd.DataFrame(data[x])) for x in ('train', 'test', 'val')]
trees = dtecDF_Itrain.drop(['Street_Type','Common_Name','Genus','Species','Street_Suffix','Longitude','Latitude','extended'], axis=1).to_dict('records')
trees[0]['lat']

34.146522093647505

time: 329 ms (started: 2022-11-18 03:04:53 +00:00)


In [ ]:
cd ..

/content/drive/MyDrive/Uni/Thesis/Repo
time: 4.86 ms (started: 2022-11-18 02:16:21 +00:00)


In [ ]:
from utils import *
import scipy

CAFFE_DIR = '/content/drive/MyDrive/Uni/Thesis/Repo/caffe'
EXTRACTORS = { 'vgg' : { 'model_file' : CAFFE_DIR+'/models/vgg_ilsvrc_16/VGG_ILSVRC_16_layers.caffemodel',
                           'deploy_prototxt' : CAFFE_DIR+'/models/vgg_ilsvrc_16/deploy.txt',
                           'batch_size' : 10, 'layer' : 'fc7', 'mean' : np.asarray((104,117,123)), 'channel_swap' : (2,1,0), 
                           'raw_scale' : 1.0, 'input_scale' : None },
               'alexnet' : { 'model_file' : CAFFE_DIR+'/models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                             'deploy_prototxt' : CAFFE_DIR+'/models/bvlc_reference_caffenet/deploy.prototxt',
                             'batch_size' : 10, 'layer' : 'fc6', 'mean' : np.asarray((104,117,123)), 'channel_swap' : (2,1,0), 
                             'raw_scale' : 255.0, 'input_scale' : None } 
}

# Feature extractor that concatenates CNN features from the applicable aerial 
# image and the closest street view image.  
def extract_tree_features(dataset, trees, feature_type='alexnet', gpu_device=0):
  bsz = EXTRACTORS[feature_type]['batch_size']
  for i in range(0, len(trees), bsz):
    aerial_images = [dataset.aerial_extractor.extract_image(trees[j]['lat'], trees[j]['lng'], 256, 256) for j in range(i,min(len(trees),i+bsz))]
    aerial_feats = cnn_feature_extractor(aerial_images, EXTRACTORS[feature_type], gpu_device=gpu_device)
  
    streetview_images = [dataset.streetview_extractor.extract_image(dataset.streetview_extractor.get_nearest_pano(trees[j]['lat'], trees[j]['lng']), trees[j]['lat'], trees[j]['lng'], (8,8), height=-1)  for j in range(i,min(len(trees),i+bsz))]
    streetview_feats = cnn_feature_extractor(streetview_images, EXTRACTORS[feature_type], gpu_device=gpu_device)
    
    if i == 0: X = np.zeros((len(trees), aerial_feats.shape[1]+streetview_feats.shape[1]))
    X[i:min(len(trees),i+bsz),0:aerial_feats.shape[1]] = aerial_feats
    X[i:min(len(trees),i+bsz),aerial_feats.shape[1]:] = streetview_feats
  return X

import caffe
g_caffe_net, g_caffe_transformer = None, None
def cnn_feature_extractor(images, params, gpu_device = 0):
  global g_caffe_net, g_caffe_transformer
  if g_caffe_net is None:
    g_caffe_net = caffe.Net(params['deploy_prototxt'], params['model_file'], caffe.TEST)
    g_caffe_transformer = caffe.io.Transformer({'data': g_caffe_net.blobs['data'].data.shape})
    if gpu_device >= 0:
      caffe.set_device(gpu_device)
      caffe.set_mode_gpu()

  net, transformer = g_caffe_net, g_caffe_transformer
  transformer.set_mean('data', params['mean'])
  transformer.set_transpose('data', (2,0,1))
  if not params['input_scale'] is None: transformer.set_input_scale('data', params['input_scale'])
  if not params['raw_scale'] is None: transformer.set_raw_scale('data', params['raw_scale'])
  if not params['channel_swap'] is None: transformer.set_channel_swap('data', params['channel_swap'])
  net.blobs['data'].data[0:len(images),...] = [transformer.preprocess('data', np.array(images[i]).astype(np.float32)) for i in range(len(images))]
  output = net.forward()
  return net.blobs[params['layer']].data[0:len(images),...].copy()

time: 10.2 ms (started: 2022-11-18 04:49:28 +00:00)


In [ ]:
import cv2
print(cv2.__version__)
import caffe

4.6.0


ModuleNotFoundError: ignored

time: 21.2 ms (started: 2022-11-18 08:04:33 +00:00)


In [ ]:
!nvidia-docker run -ti bvlc/caffe:gpu caffe --version

/bin/bash: nvidia-docker: command not found
time: 248 ms (started: 2022-11-18 04:49:38 +00:00)


In [ ]:
X = extract_tree_features(pDs, trees[0:2])

AttributeError: ignored

time: 46.9 ms (started: 2022-11-18 04:49:30 +00:00)


In [ ]:
len(trees)